# Gradient Checking

**Problem Statement**: Build a model to detect fraud. Whenever someone makes a payment, you want to see if the payment might be fraudulent, such as if the user's account has been taken over by a hacker. Use gradient checking as proof that your backpropagation is actually working.

In [1]:
""" PACKAGES """
import numpy as np
from testCases import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

### The Process of Gradient Checking

Backpropagation computes the gradients $\frac{\partial J}{\partial \theta}$, where $J$ is computed using forward propagation and your loss function. Through these, the code used to compute $J$ can be used to verify the code for computing $\frac{\partial J}{\partial \theta}$. 

The definition of a derivative (or gradient):
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

We use the definition of a derivative to make sure we compute $\frac{\partial J}{\partial \theta}$ correctly.

### 1D gradient checking

1. Start with $x$, then evaluate the function $J(x)$ ("forward propagation"). 
2. Compute the derivative $\frac{\partial J}{\partial \theta}$ ("backward propagation").

In [6]:
""" 1D FORWARD PROPAGATION: COMPUTE J """
def forward_propagation(x, theta):
    J = theta * x
    return J

""" 1D BACKWARD PROPAGATION: COMPUTE dJ/dtheta """
def backward_propagation(x, theta):
    dtheta = x
    return dtheta

**Gradient Checking**:
1. Compute "gradapprox" using the definition of a derivative and a small value of $\varepsilon$.
2. Compute "grad" using backward propagation
3. Compute the relative difference between "gradapprox" and the "grad" using the formula:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$

If this difference is small (say less than $10^{-7}$), your gradient is probably correct. Otherwise, there may be a mistake in the gradient computation. 


In [7]:
""" 1D GRADIENT CHECK """
def gradient_check(x, theta, epsilon = 1e-7):
    
    # gradapprox
    thetaplus = theta + epsilon
    thetaminus = theta - epsilon
    J_plus = forward_propagation(x, thetaplus)
    J_minus = forward_propagation(x, thetaminus)
    gradapprox = (J_plus - J_minus) / (2*epsilon)
    
    # grad
    grad = backward_propagation(x, theta)
    
    # difference
    numerator = np.linalg.norm(grad-gradapprox)                       
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator
    
    if difference < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference

### N-dimensional gradient checking

In [9]:
""" FORWARD PROPAGATION WITH N DIMENSIONS """
def forward_propagation_n(X, Y, parameters):
    
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [13]:
""" BACKWARD PROPAGATION WITH N DIMENSIONS """
def backward_propagation_n(X, Y, cache):
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

**How does gradient checking work?**.

Here, $\theta$ is not a scalar anymore, but rather a dictionary "parameters". The function "`dictionary_to_vector()`" converts "parameters" into a vector "values", obtained by reshaping all parameters (W1, b1, W2, b2, W3, b3) into vectors and concatenating them. The inverse function "`vector_to_dictionary`" outputs back "parameters". "`gradients_to_vector()`" converts "gradients" into a vector "grad".

In [11]:
""" GRADIENT CHECK WITH N DIMENSIONS """
def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    
    # Set up variables
    parameters_values, _ = dictionary_to_vector(parameters) # we only care about the first value
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # gradapprox
    for i in range(num_parameters):
        
        # J plus
        thetaplus = np.copy(parameters_values)
        thetaplus[i][0] += epsilon
        J_plus[i], _ =  forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))
        
        # J minus
        thetaminus = np.copy(parameters_values)
        thetaminus[i][0] -= epsilon
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus))
        
        # Gradapprox
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
    
    # difference
    numerator = np.linalg.norm(grad-gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator

    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [14]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

Your backward propagation works perfectly fine! difference = 1.18855520355e-07


**Notes** 
- Gradient Checking is slow and computationally costly, so we don't run gradient checking at every iteration during training.
- Gradient Checking doesn't work with dropout. Run gradient check without dropout to make sure backprop is correct, then add dropout.